### Import Some Neccesary Library

In [1]:
import cv2

from utils import *
import warnings
import numpy as np
from matplotlib import pyplot as plt

warnings.filterwarnings('ignore')

In [2]:
def image_resizer(filePath, new_width):
    image = cv2.imread(filePath)
    height = image.shape[0]
    width = image.shape[1]
    
    ratio = new_width / width
    new_height = int(height * ratio)
    
    dimensions = (new_width, new_height)
    new_image = cv2.resize(image, dimensions, interpolation=cv2.INTER_LINEAR)
    return new_image
    
    # if you see some info, uncommit below codes
    # print("New shape:      ", new_image.shape)
    # cv2.imshow("Resized image", new_image)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [3]:
h_theta = np.pi / 180
h_lines = None
h_srn = 0  # divisor for rho
h_stn = 0  # divisor for theta

# Canny
c_t1 = 50
c_t2 = c_t1 * 3

# Process model image=========================
model_img = cv.imread('images/0.jpg')
model_gray = cv.cvtColor(model_img, cv.COLOR_BGR2GRAY)
cv.imwrite("./output/model_gray_scale.jpg", model_gray)
model_canny = cv.Canny(model_gray, c_t1, c_t2)
cv.imwrite("./output/model_canny_detected.jpg", model_gray)
model_lines = cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR)
source_lines = cv.HoughLines(model_canny, 1.1, h_theta, 247, h_lines, h_srn, h_stn)

linesP = cv.HoughLinesP(model_canny, 1, np.pi / 180, 50, None, 50, 10)
cdstP = np.copy(cv.cvtColor(model_canny, cv.COLOR_GRAY2BGR))

if linesP is not None:
    for i in range(0, len(linesP)):
        l = linesP[i][0]
        cv.line(cdstP, (l[0], l[1]), (l[2], l[3]), (0,0,255), 3, cv.LINE_AA)

cv.imwrite("./output/model_hough_lines.jpg", cdstP)

cartesian_lines = []
if source_lines is not None:
    for i in range(0, len(source_lines)):
        pt1, pt2 = add_line(source_lines, i, model_lines)
        cartesian_lines.append([[pt1[0], pt1[1]], [pt2[0], pt2[1]]])
        
# print("cartesian_lines:", cartesian_lines)
all_intersections = junctions(cartesian_lines)
print("all_intersections:", all_intersections)
cluster_centers = kmeans_centers(all_intersections, 10)

mark_all(cluster_centers, model_lines)
cv.imwrite("model_lines.jpg", model_lines)

# Calculate corners
m_top, m_bot = get_model_corners(cluster_centers)
print("Cluster centers:", cluster_centers)
print("Points on top:", m_top)
print("Points on bot:", m_bot)
model_joined_topbot = m_top + m_bot

In [5]:
image_file = './images/1_man.jpg'
default_width = 700
frame_raw = cv.imread(image_file)

if frame_raw.shape[1] > default_width:
    new_image = image_resizer(image_file, default_width)
    # cv.imwrite("7_scaler.jpg", new_image)
    frame_raw = cv.cvtColor(new_image, cv.COLOR_BGR2GRAY)

src = frame_raw
dst = cv.Canny(src, 50, 200, None, 3)
cdst = cv.cvtColor(dst, cv.COLOR_GRAY2BGR)

cv.imwrite("input-gray-image.jpg", cv.cvtColor(frame_raw, cv.COLOR_BGR2GRAY))
cv.imwrite("input-canny.jpg", cdst)

frame_lines = dst
cartesian_lines = []
lines = cv.HoughLines(dst, 1, np.pi / 180, 110, None, 0, 0)

#Draw hough Lines

if lines is not None:
    for i in range(0, len(lines)):
        rho = lines[i][0][0]
        theta = lines[i][0][1]
        a = math.cos(theta)
        b = math.sin(theta)
        x0 = a * rho
        y0 = b * rho
        pt1 = (int(x0 + 1000*(-b)), int(y0 + 1000*(a)))
        pt2 = (int(x0 - 1000*(-b)), int(y0 - 1000*(a)))
        cv.line(cdst, pt1, pt2, (0,0,255), 3, cv.LINE_AA)
        
cv.imwrite("input-image-hough-lines.jpg", cdst)

if lines is not None:
    for i in range(0, len(lines)):
        pt1, pt2 = add_line(lines, i, frame_lines)
        cartesian_lines.append([[pt1[0], pt1[1]], [pt2[0], pt2[1]]])

junction_points = junctions(cartesian_lines)
junction_points_normalized = []

for junction_point in junction_points:
    if (junction_point[0] > 0 and junction_point[1] > 0):
        if (junction_point[0] < src.shape[1] and junction_point[1] > src.shape[0]):
            junction_points_normalized.append(junction_point)

cluster_centers = kmeans_centers(junction_points, 25)
mark_all(junction_points, frame_lines, rbg=(0, 0, 255))

printed = ''
for cluster_center in cluster_centers:
    printed += str(cluster_center) + ', '

print(printed)

warped_img = []
n_noninf = get_noninf(cluster_centers)
if (len(n_noninf) >= 4):
    top, bot = get_frame_corners(n_noninf)
    frame_joined_topbot = top + bot
    if (len(frame_joined_topbot) == 4):
        # Calculate the mask for the warping
        mask, status = cv.findHomography(np.array(frame_joined_topbot), np.array(model_joined_topbot))
        print("Mask:\n", mask)
        # Get frame resolution for cv.warpPerspective()
        height, width, channels = model_img.shape
        warped_img = cv.warpPerspective(frame_raw, mask, (width, height))

    mark_all(n_noninf, frame_lines)
    print("Clusters: ", len(n_noninf))  # Number of cluster centers on screen

cv.imshow("Model", warped_img)
# cv.imwrite("./output/warped_image.jpg", warped_img)
cv.imwrite("warped_image.jpg", warped_img)
cv.waitKey(0)
cv.destroyAllWindows()

# 6_man
# 14,280 +   [ 26.36708861, 161.91139241]
# 470,463 +  [467.25       407.37068966]
# 620,122 +  [601.65934066 134.13186813]
# 414,49 +  [371.79054054  45.22297297]
# cv.imwrite("warper.jpg", warped_img)